In [1]:
## Install
!pip install easyocr

In [2]:
## Imports

import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np
import os
import csv
import io

import torch
import cv2
import easyocr

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# easyocr loading
reader = easyocr.Reader(['en']) 

In [7]:
%cd /home/studio-lab-user/DS-Projects/OCR

/home/studio-lab-user/DS-Projects/OCR


In [8]:
# # OCR_images에 있는 리스트들로 컬럼 하나 만들기. 리스트 이름 따기.
# import pandas as pd
# import os
# image_df = pd.DataFrame(os.listdir(), columns=['image'])

In [9]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("easy_results.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


In [10]:
image_df = data[['image','relevant']]

In [11]:
image_df

,image,relevant
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1
1,68ece3fab138feb537eee296f3348f3543e23a1d,0
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0
3,a278812a832f24a822e2602b91a7b1f5749926cb,1
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0
...,...,...
464,43a18980d983e7835ce8a9f50ae8953390197f73,0
465,a2db2c697881750d70feaa1a90d4f271338b9337,0
466,077d6694396e202064fb5b01b42837d2949e4ce7,0
467,a8057438d83d4fc141347237c23e2d3fc714d43f,0


In [12]:
image_list = image_df['image'].tolist()

In [13]:
image_list

['...',
 'empty',
 'cfaccfa730fca29e0df6f2e18b74735951d18428',
 '7643759f30a2445e8084ecf848628b8ae7d81a92',
 'fc06cacb287cdea9a186109e792b1cc3b34c97ab',
 'f44b5b470423248e15cb5a28a1200311af2a9c1b',
 '1818410beb2f12283a6ab72db3b988ae56fd3b3e',
 '2c64ca51b53c48fcaf8630d927dc53a4a74a14bf',
 '777f9539646b03a257753ec475c9d2793b0c86ab',
 '02e84e0140c449cf60e451672f210198594ac620',
 '2a1cb620245773522750665f9c99cbc5ef61aebb',
 '3910618b165634d36452bd8d32a6c792ee728150',
 '4111f60e1607757c6f829638268aefe602f33ba7',
 'e7548d35100cf01b8e9dbd44d58163a01632c446',
 '14f4e5198283fbe1a1ffae046ef359af50baee30',
 '6528ef6a197b0c379b1af5afeb33632c151ed814',
 '9a6791a009f973e38fccd9ed56711a79a95ba6d8',
 'd8fdca3761d5fd3a3f99a8aeaeb907107f6a71a3',
 '1e4262b940e28e48b0a674611114884af56a3afe',
 'd0a8cf4cd898ba8c4bc1435a08b0f7f702860b0f',
 '43a18980d983e7835ce8a9f50ae8953390197f73',
 'a2db2c697881750d70feaa1a90d4f271338b9337',
 '077d6694396e202064fb5b01b42837d2949e4ce7',
 'a8057438d83d4fc141347237c23e2d3fc71

In [14]:
# words and scores
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    
    img_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    # easyocr readtext
    img_text = reader.readtext(img_path)
    #empty lists
    words = []
    confidences = []
    #appending results into the lists
    for (bbox, text, prob) in img_text:
        words.append(text)
        confidences.append(prob)
    result = [words, confidences]
    return result

In [35]:
# %cd /home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images

/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images


In [15]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_df

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[NG US], [0.9635912292490354]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[U, Zfsz er alibaba corfi], [0.03421223296994..."
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[Impostor], [0.995059332902207]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[Press t0 make a sound, 0, 6], [0.77249010857..."
...,...,...,...
464,43a18980d983e7835ce8a9f50ae8953390197f73,0,"[[Emergei], [0.5969751995558676]]"
465,a2db2c697881750d70feaa1a90d4f271338b9337,0,"[[Moww""t:, 2022, Bski:, 2 0 2 2, 2 0 2 2], [0...."
466,077d6694396e202064fb5b01b42837d2949e4ce7,0,"[[ShhhhhHh?], [0.528170395393331]]"
467,a8057438d83d4fc141347237c23e2d3fc714d43f,0,"[[], []]"


In [16]:
import re
import string
def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [17]:
image_df["words"] = image_df.apply(clean_text, axis=1)

In [18]:
## define the target words / phrases

targets = ["among us", "impostor"]

In [19]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)


def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [24]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            print(new_sample)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score

In [25]:
import itertools
import nltk
import numpy as np

image_df["OCR_score"] = image_df.apply(get_final_score, axis=1)

ng us
u zfsz er alibaba corfi
u
zfsz er alibaba corfi
impostor
press t0 make a sound 0
0 6
press t0 make a sound
0
6
 mong us us
us look
look yau
yau look
look 5u5
5u5 m
m  yong 
 yong   aong  
 aong   trust
 mong us
us
look
yau
look
5u5
m
 yong 
 aong  
trust
8 ebengiog
ebengiog fece
8
ebengiog
fece
3  
3
 
09
salvya
gia2 3
3  
  1
1 8
gia2
3
 
1
8
u

reversible 
9 6
9
6
  3
 
3
6 feee
feee chenging
chenging fcee
fcee chenging
chenging fece
fece 3
3 chengioa
chengioa chenaina
chenaina fece
fece fece
6
feee
chenging
fcee
chenging
fece
3
chengioa
chenaina
fece
fece
l  92
 92 9
9 073491
073491  €
 € 7g
7g  
  08
08 1
1 88
88 l
l  8
 8  970
 970  dx
 dx iia
iia  
l
 92
9
073491
 €
7g
 
08
1
88
l
 8
 970
 dx
iia
 
1 1
1 1
1 ble
ble bla
bla 3obix
3obix blp
blp rpbeez
rpbeez 8
8 1
1 0 pyc7
0 pyc7 bld
bld fpyj
fpyj 1
1  c28
1
1
1
ble
bla
3obix
blp
rpbeez
8
1
0 pyc7
bld
fpyj
1
 c28
impostor 3
3  
  1
1 8
impostor
3
 
1
8
100380
luong  3
3 g
g 1
1 j
luong 
3
g
1
j
us 9
9 q5
q5 8388
8388 ahb l
a

In [26]:
image_df = image_df.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
image_df

,image,relevant,words,OCR_score
438,8b3790e98660f3b1949f77ac42542169519f23c1,1,"[[impostor], [0.999980738813395]]",0.999981
242,d8ed68c8657219f8e11febbf689adf8889e5a7ad,1,"[[impostor, 6, 1, i, 3], [0.9999702862889432, ...",0.999970
132,ae008c50c09aaee6babced0b3cd5b6551c32d82d,1,"[[impostor, 6, 1, i, 3], [0.9999702862889432, ...",0.999970
280,63094b2fcfe46bc394901b231968251c43b3e876,1,"[[impostor, 6, 1, i, 3], [0.9999702862889432, ...",0.999970
227,3fe8539d691409d2207cf5f2bd5ce453c79fe185,1,"[[ mong us, impostor, mong us, impos, tor], [...",0.999951
462,1e4262b940e28e48b0a674611114884af56a3afe,1,"[[gbmong us, , impostor], [0.1876039987647094...",0.999933
53,75f467a04f86432f8e752744a486c415e7270c9b,1,"[[gbmong us, impostor], [0.16595820194336824, ...",0.999929
387,304a52cba431f97dba26cac253f3fc8f5200efd6,1,"[[impostor], [0.9998762536812379]]",0.999876
69,0fdcf798e38df01a52ac0ad5947f4222a64859e4,1,"[[impostor, 1, 1, 3], [0.9998564857150571, 0.8...",0.999856
229,e0614be54bfaf19965434fef71f85b18c9a46f1c,1,"[[impostor, s, 1, 1, 3], [0.999843545828632, 0...",0.999844


In [27]:
RA_score = roc_auc_score(image_df["relevant"], image_df["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

Rekognition ROC AUC Score =  0.88


In [28]:
# Dataframe of true positives

tp = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] >= 0.5)] 
print(len(tp))

46


In [29]:
# Dataframe of true negatives

tn = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] < 0.5)] 
print(len(tn))

318


In [30]:
# Dataframe of false positives

fp = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] >= 0.5)] 
print(len(fp))

0


In [32]:
# Dataframe of false negatives

fn = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] < 0.5)] 
print(len(fn))

105


In [33]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

1.0


In [34]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

0.304635761589404


In [35]:
fn

,image,relevant,words,OCR_score
321,c7a40481652e38221fdc8d4dc14521a21b89fa96,1,"[[i, o, killer , 1igcussi , mv, crey, stor, ...",0.499918
107,0d88b1376acd2c9b69ad3a509d811cac4272434e,1,"[[reading th, lm, ndal, wh didd, mposio, impo...",0.490325
440,b9862cccf436619cb436886cbda61bcbeb61b28b,1,"[[impostor], [0.4863055894365148]]",0.486306
281,7681abd1a0bb570d460989b8c0c6cc853f251823,1,"[[among us , 6, , , 3], [0.4753841035739426,...",0.444839
215,3da262c4af82a910bae23f89874faf46f7e9d4f8,1,"[[k, o, among us, 52, , 1, 1, 3], [0.2650408...",0.415269
254,fc45449106e2327b8952f5cfc5be1ccb155c14e2,1,"[[us, mong us, 6, mong, 3, 3, 3], [0.9020726...",0.409001
230,51416eb364f8e8c4acbdda341d6666841ed30de1,1,"[[gis, g, 9, among us, 9], [0.1850589523537109...",0.381545
424,b8019c2cad1fb1584afefcb65b65b912725e011e,1,"[[tor, ailgus, yov ye, dumbi, susii ], [0.999...",0.374771
68,43652e597d82e50337e905c1246a8083b942a5aa,1,"[[that s ki, sus, tho, sim, 303, u, sabotac, ...",0.364912
98,1f677c2708856bb52f9180481ff7c0b038d4bbe5,1,"[[uo, 8, us, is, us, qnc, us, us, us, us, vg u...",0.354041


In [41]:
image_df.to_csv('/home/studio-lab-user/DS-Projects/OCR/easy_ocr.csv')